In [7]:
import pandas as pd

df = pd.read_csv('./generated_input_data.csv', encoding='UTF-8', low_memory=False)

# display(df)

subject = '기후 데이터 기반의 도시 열섬 현상 분석'
description = '위성 원격탐사 데이터와 국내 기상 관측소 데이터를 결합하여 도시 열섬 현상의 시공간적 패턴을 분석하고, 인공지능 기반 예측 모델을 통해 기후 변화 대응 전략을 모색하는 연구'

In [8]:
from langgraph.graph import MessagesState

class State(MessagesState):
    subject: str = ''
    description: str = ''
    data: pd.DataFrame = pd.DataFrame()
    

In [9]:
# Schema
from typing_extensions import Annotated
from pydantic import Field, BaseModel

class IDRelevance(BaseModel):
    relevant_id: Annotated[
        list[int],
        Field(
            ..., 
            max_items=5, 
            min_items=3,
            description="가장 관련성이 높은 데이터 ID 리스트, 길이 최소 3개/최대 5개", 
        )
    ]


In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate

# 프롬프트
relevance_template = '''
당신은 데이터 과학자입니다. 아래는 연구 데이터 목록입니다.

각 데이터 항목은 다음 컬럼을 가지고 있습니다:
- ID: 각 데이터의 고유키
- 구분: 논문 또는 데이터셋
- 제목
- 설명
- 작성일
- 해외/국내
- 작성언어
- 키워드

[목표]
주어진 연구 주제와 가장 관련성 높은 3개의 데이터 항목을 선택하세요. 
- 관련성 기준: 제목, 설명, 키워드에 기반

[Input]
연구 주제: {subject}
연구 설명: {description}

[Data]
데이터 목록:
{data}

[Output]
가장 관련성 높은 3개 항목의 ID를 JSON으로 출력해주세요.
'''

relevance_prompt = PromptTemplate.from_template(relevance_template)
prompt = relevance_prompt.invoke(
    {
        'subject': subject, 
        'description': description,
        'data': df.to_dict(orient="records"),
    }
)

# sLLM
sllm = ChatOpenAI(model='gpt-4o-mini', temperature=0)

structured_sllm = sllm.with_structured_output(IDRelevance)
res = structured_sllm.invoke(prompt)

# Output
print(res.relevant_id)


[25, 14, 28]
